In [1]:
# imports
import json
import pandas as pd
import os
import re
import datetime

In [18]:
def salva_capitolo(capitolo, numero, cartella):
    """Salva un singolo capitolo in un file."""
    nome_file = f"capitolo_{numero:02d}.txt"
    percorso_file = os.path.join(cartella, nome_file)

    with open(percorso_file, 'w', encoding='utf-8') as f:
        f.write(f"CAPITOLO {numero}\n")
        f.write("=" * 50 + "\n\n")
        f.write(capitolo)


def carica_testo_da_file(percorso_file):
    """Carica il testo da un file."""
    try:
        with open(percorso_file, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Errore: File '{percorso_file}' non trovato.")
        return None
    except Exception as e:
        print(f"Errore nel leggere il file: {e}")
        return None


def dividi_testo_in_capitoli(testo, min_caratteri=2000, max_caratteri=5000, file_input="testo.txt", cartella_output="capitoli", gestione_ultimo="unisci"):
    """
    Divide un testo lungo in capitoli di lunghezza specificata.

    Args:
        testo (str): Il testo da dividere
        min_caratteri (int): Minimo caratteri per capitolo (default: 10000)
        max_caratteri (int): Massimo caratteri per capitolo (default: 20000)
        file_input (str): Nome del file di input (per riferimento)
        cartella_output (str): Nome della cartella dove salvare i capitoli
        gestione_ultimo (str): Come gestire l'ultimo capitolo se troppo corto
            - "unisci": unisce al capitolo precedente (default)
            - "separato": mantiene separato anche se corto
            - "soglia": unisce solo se sotto una certa soglia

    Returns:
        list: Lista dei capitoli creati
    """

    # Crea la cartella di output se non esiste
    if not os.path.exists(cartella_output):
        os.makedirs(cartella_output)

    # Dividi il testo in frasi (cerca punti seguiti da spazio e maiuscola)
    frasi = re.split(r'(?<=[.!?])\s+(?=[A-Z])', testo)

    capitoli = []
    capitolo_corrente = ""
    numero_capitolo = 1

    for frase in frasi:
        # Calcola la lunghezza se aggiungessimo questa frase
        lunghezza_proposta = len(capitolo_corrente) + len(frase) + 1

        # Se siamo sotto il minimo, aggiungi sempre
        if len(capitolo_corrente) < min_caratteri:
            capitolo_corrente += frase + " "

        # Se siamo sopra il massimo, chiudi il capitolo
        elif lunghezza_proposta > max_caratteri:
            # Salva il capitolo corrente
            capitolo_pulito = capitolo_corrente.strip()
            if capitolo_pulito:
                capitoli.append(capitolo_pulito)
                # salva_capitolo(capitolo_pulito, numero_capitolo, cartella_output)
                # salva_capitolo(capitolo_pulito, numero_capitolo, cartella_output)
                numero_capitolo += 1

            # Inizia nuovo capitolo
            capitolo_corrente = frase + " "

        # Se siamo nella zona ottimale, aggiungi
        else:
            capitolo_corrente += frase + " "

    # Gestisci l'ultimo capitolo
    capitolo_pulito = capitolo_corrente.strip()
    if capitolo_pulito:
        # Calcola la soglia per considerare un capitolo "troppo corto"
        soglia_minima = min_caratteri * 0.5 if gestione_ultimo == "unisci" else min_caratteri * 0.3

        if gestione_ultimo == "separato":
            # Mantieni sempre separato
            capitoli.append(capitolo_pulito)
            

        elif len(capitolo_pulito) < soglia_minima and capitoli:
            # Unisci al capitolo precedente
            # Rimuovi l'ultimo file salvato
            ultimo_file = f"capitolo_{numero_capitolo-1:02d}.txt"
            ultimo_percorso = os.path.join(cartella_output, ultimo_file)
            if os.path.exists(ultimo_percorso):
                os.remove(ultimo_percorso)

            # Rimuovi l'ultimo capitolo dalla lista
            ultimo_capitolo = capitoli.pop()
            numero_capitolo -= 1

            # Unisci con il nuovo testo
            capitolo_unito = ultimo_capitolo + "\n\n" + capitolo_pulito
            capitoli.append(capitolo_unito)

            # Risalva il capitolo unito
            #  salva_capitolo(capitolo_unito, numero_capitolo, cartella_output)
        else:
            # Salva come capitolo separato
            capitoli.append(capitolo_pulito)
            # salva_capitolo(capitolo_pulito, numero_capitolo, cartella_output)

    return capitoli

def creazione_dataset(n):
  testi = []
  for i in n:
    print(f"diviso il transcript{i}")
    with open(f'..\\Documents\\Transcripts\\transcript{i}.json', 'r') as f:
        data = json.load(f)

    testi.append(data.get('transcription'))
    X_capitoli = [dividi_testo_in_capitoli(x) for x in testi]
    capitoli_flat = [capitolo for testi in X_capitoli for capitolo in testi]
    df_final = pd.DataFrame({"Text": capitoli_flat})
  return df_final

In [3]:
# definisci il percorso del transcript che vuoi prendere con {i} finale, esempio:
# percorso_transcript = f'transcripts\\transcript{i}.json'


In [20]:
# n dei transcripts
n = [10] # i transcript devono esistere
final_dataset= creazione_dataset(n)
percorso = "..\\Documents\\Datasets\\ciaociao.csv"
final_dataset.to_csv(percorso, index=False, encoding='utf-8')

diviso il transcript10
